In [2]:
import ee
import geemap
ee.Initialize()

In [3]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)

# Define a region in which to generate a sample of the input.
region = geometry = ee.Geometry.Polygon(
        [[[-46.31633300781251, -12.379228237680417],
          [-45.31108398437501, -12.298734131305821],
          [-45.30009765625001, -11.589339623793398],
          [-46.33830566406251, -11.605482676116234]]])

Map

Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [4]:
def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': 250,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':250,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':250,
        'bestEffort':True,
        'maxPixels':1000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':250,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':250,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':250,
    'bestEffort':True,
    'maxPixels':1000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    

In [34]:
import requests
import pandas as pd
import json
import time
#import os

#out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')


In [37]:
stats = list();

In [38]:
#Dividir o processamento das imagens de acordo com o ano para não sobrecarregar o sistema
ano = list(range(2000, 2022))

string_ints = [str(int) for int in ano]
#stats = list();
    
for i in (string_ints):
    
    print (i)
    #Filtrar a territorialidade 
    #4 principais meses de safra
    #filter start and end date   
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( i + '-01-01', i +'-04-30').filter(ee.Filter.calendarRange(1, 4,'month')).map(clusterization)
        
    #Pegar a propriedade que nos interessa em cada imagem - o EVI_Cluster Calculado 
    a = mcollection.aggregate_array('EVI_Cluster')
    
    
    #10s de soneca pra o GEE não derrubar as requests
    time.sleep(10)
    # Fazer a informação passar do servidor do google para o jupyter
    x = a.getInfo()
    #Salvar cada ano
    stats.append(x)    
    

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [46]:
stats

[[6874.760045582916,
  6394.279193473026,
  6123.178791068757,
  4253.107085775173,
  3838.8418424337997],
 [5259.268694765466,
  6258.676460451491,
  7031.668107173725,
  7296.111597511022,
  6880.736273560307,
  6378.035850757803,
  4772.181428806765,
  3973.964507186109],
 [6423.580114523686,
  8020.402897702327,
  7868.0459846516405,
  7133.997767013639,
  4857.447993629165,
  4072.496886600407,
  3881.0738598991447,
  3893.3381385241637],
 [6578.208821502412,
  7070.053477588872,
  7292.763609272671,
  7051.363568286062,
  6780.649959930546,
  5283.531318867605,
  4649.450547213031,
  4371.640041696527],
 [6718.109842898365,
  7036.585586831731,
  7988.7452463088175,
  7543.445754792976,
  6559.527911185355,
  5558.025200310158,
  4846.26362085308,
  4601.5968270055355],
 [5862.068056060422,
  6604.773912058024,
  7686.325467900986,
  7541.497537749966,
  7247.702174746968,
  6307.824136858476,
  5332.14668510352,
  5279.069751307504],
 [6061.925739081352,
  5825.8075855379775,
  

In [41]:
df = pd.DataFrame({'lin':stats})

In [43]:
#Separate comma
#df = df.apply(lambda x:pd.Series(x))

In [57]:
df = pd.DataFrame(stats, columns = ['EVI1', 'EVI2','EVI3','EVI4','EVI5','EVI6','EVI7','EVI8'])

In [58]:
df

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842,NaN,NaN,NaN
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787


In [60]:
#Reshape da 1a linha - Imagens do MODIS a partir de Mar/2000
new_order =  ['EVI6', 'EVI7', 'EVI8','EVI1','EVI2','EVI3','EVI4','EVI5'] # specify new order of the third row
i = 0 # specify row number
df.loc[0] = df.loc[0, new_order].values

In [64]:
#Corrigir os Nans
#Features dos EVIS pronta
df.fillna(0)


,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,0.000000,0.000000,0.000000,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787


In [141]:
GEOCODE = '2919553'


In [142]:
url = f'https://apiprevmet3.inmet.gov.br/estacao/proxima/{GEOCODE}'
response = requests.get(url).json()
response

{'estacao': {'UF': 'BA',
  'CODIGO': 'A402',
  'LONGITUDE': '-45.02694443',
  'REGIAO': 'NE',
  'DISTANCIA_EM_KM': '105',
  'NOME': 'BARREIRAS',
  'LATITUDE': '-12.12472221',
  'GEOCODE': '2903201'},
 'dados': {'DC_NOME': 'BARREIRAS',
  'PRE_INS': None,
  'TEM_SEN': None,
  'VL_LATITUDE': '-12.12472221',
  'PRE_MAX': None,
  'UF': 'BA',
  'RAD_GLO': None,
  'PTO_INS': None,
  'TEM_MIN': None,
  'VL_LONGITUDE': '-45.02694443',
  'UMD_MIN': None,
  'PTO_MAX': None,
  'VEN_DIR': None,
  'DT_MEDICAO': '2021-07-22',
  'CHUVA': None,
  'PRE_MIN': None,
  'UMD_MAX': None,
  'VEN_VEL': None,
  'PTO_MIN': None,
  'TEM_MAX': None,
  'VEN_RAJ': None,
  'TEM_INS': None,
  'UMD_INS': None,
  'CD_ESTACAO': 'A402',
  'HR_MEDICAO': '1700'}}

In [143]:
estacao = response['dados']['CD_ESTACAO']

In [144]:
new_url = f'https://apitempo.inmet.gov.br/estacao/diaria/1999-11-01/2021-04-30/{estacao}'
json_response = requests.get(new_url).json()
dados_estacao = pd.DataFrame(json_response)
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE
0,None,1999-11-01,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
1,None,1999-11-02,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
2,None,1999-11-03,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
3,None,1999-11-04,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
4,None,1999-11-05,BARREIRAS,None,None,None,-12.12472221,None,None,BA,None,A402,-45.02694443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,86.9,2021-04-26,BARREIRAS,56,24,None,-12.12472221,19.5,30.8,BA,0.7,A402,-45.02694443
7848,88.3,2021-04-27,BARREIRAS,59,24.4,None,-12.12472221,20.8,31,BA,0.6,A402,-45.02694443
7849,84.8,2021-04-28,BARREIRAS,50,24.9,None,-12.12472221,21.1,32.4,BA,1,A402,-45.02694443
7850,80.3,2021-04-29,BARREIRAS,45,25.5,None,-12.12472221,20.5,32.2,BA,0.7,A402,-45.02694443


In [145]:
dados_estacao['CHUVA'] = dados_estacao['CHUVA'].astype(float)
dados_estacao['DT_MEDICAO'] = pd.to_datetime(dados_estacao['DT_MEDICAO'])
dados_estacao['ANO'] = dados_estacao['DT_MEDICAO'].dt.year
dados_estacao['MES'] = dados_estacao['DT_MEDICAO'].dt.month
dados_estacao

,UMID_MED,DT_MEDICAO,DC_NOME,UMID_MIN,TEMP_MED,CHUVA,VL_LATITUDE,TEMP_MIN,TEMP_MAX,UF,VEL_VENTO_MED,CD_ESTACAO,VL_LONGITUDE,ANO,MES
0,None,1999-11-01,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,1999,11
1,None,1999-11-02,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,1999,11
2,None,1999-11-03,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,1999,11
3,None,1999-11-04,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,1999,11
4,None,1999-11-05,BARREIRAS,None,None,NaN,-12.12472221,None,None,BA,None,A402,-45.02694443,1999,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,86.9,2021-04-26,BARREIRAS,56,24,NaN,-12.12472221,19.5,30.8,BA,0.7,A402,-45.02694443,2021,4
7848,88.3,2021-04-27,BARREIRAS,59,24.4,NaN,-12.12472221,20.8,31,BA,0.6,A402,-45.02694443,2021,4
7849,84.8,2021-04-28,BARREIRAS,50,24.9,NaN,-12.12472221,21.1,32.4,BA,1,A402,-45.02694443,2021,4
7850,80.3,2021-04-29,BARREIRAS,45,25.5,NaN,-12.12472221,20.5,32.2,BA,0.7,A402,-45.02694443,2021,4


In [146]:
b  = dados_estacao.groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [147]:
mask = (dados_estacao['MES'] <= 4) | (dados_estacao['MES'] >= 11)  
b = dados_estacao.loc[mask].groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [162]:
chuva = b.values.reshape(22,6)
df2 = pd.DataFrame(chuva, columns = ['C_NOV', 'C_DEZ','C_JAN','C_FEV','C_MAR','C_ABR'])
df2

,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,23.2,0.0,0.0,27.8,31.8
3,0.0,61.4,0.0,77.2,135.4,105.4
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.8,172.6,2.2,0.4,163.2,108.2
7,46.6,0.0,0.6,0.0,0.0,17.6
8,1.0,0.4,91.2,60.6,267.2,118.2
9,137.6,183.0,219.6,60.0,160.8,202.2


In [194]:
concat_data = pd.concat([df,df2],axis=1).fillna(0)

In [195]:
concat_data

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,0.000000,0.000000,0.000000,6874.760046,6394.279193,6123.178791,4253.107086,3838.841842,0.0,0.0,0.0,0.0,0.0,0.0
1,5259.268695,6258.676460,7031.668107,7296.111598,6880.736274,6378.035851,4772.181429,3973.964507,0.0,0.0,0.0,0.0,0.0,0.0
2,6423.580115,8020.402898,7868.045985,7133.997767,4857.447994,4072.496887,3881.073860,3893.338139,0.0,23.2,0.0,0.0,27.8,31.8
3,6578.208822,7070.053478,7292.763609,7051.363568,6780.649960,5283.531319,4649.450547,4371.640042,0.0,61.4,0.0,77.2,135.4,105.4
4,6718.109843,7036.585587,7988.745246,7543.445755,6559.527911,5558.025200,4846.263621,4601.596827,0.0,0.0,0.0,0.0,0.0,0.0
5,5862.068056,6604.773912,7686.325468,7541.497538,7247.702175,6307.824137,5332.146685,5279.069751,0.0,0.0,0.0,0.0,0.0,0.0
6,6061.925739,5825.807586,7359.243333,7091.745079,7043.773647,6863.455807,6539.112422,6297.635018,0.8,172.6,2.2,0.4,163.2,108.2
7,7232.073196,7605.324347,7664.341318,7631.458098,7331.321532,6433.052025,5034.180131,5030.260769,46.6,0.0,0.6,0.0,0.0,17.6
8,4999.699570,7348.651631,7688.411261,7908.436345,8486.042040,7014.634689,6424.546801,5131.027017,1.0,0.4,91.2,60.6,267.2,118.2
9,6366.144577,7535.637696,8258.047953,8232.011604,7413.414015,6926.599555,5692.455953,6373.981787,137.6,183.0,219.6,60.0,160.8,202.2


In [ ]:
# Api PAM SIDRA/ https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124

In [207]:
url3 = 'https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124'
json_response = requests.get(url3).json()
produtividade = pd.DataFrame(json_response)
Y = produtividade['V'].iloc[1:].values.ravel()

In [208]:
#Produtividade de Soja por ano
df3 = pd.DataFrame(Y)
df3

,0
0,...
1,2040
2,1829
3,1829
4,2880
5,2759
6,2280
7,2700
8,3036
9,2550


In [209]:
#Divisão do modelo para teste e para treinamento
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(concat_data.iloc[0:20,:], df3, test_size=(0.3))

In [221]:
#Importamos o classificador KNN e aplicamos o modelo com k vizinhos com base em X_train e Y-train

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=4)
classifier.fit(X_train, y_train)

C:\Users\Guga\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=4)

In [211]:
#Y previsto com base no classificador treinado e X_test como input

y_pred = classifier.predict(X_test)

In [212]:
#Métricas de acurácia do sklearn

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]]
              precision    recall  f1-score   support

         ...       0.00      0.00      0.00         1
        1829       0.33      1.00      0.50         1
        2550       0.00      0.00      0.00         0
        2580       0.00      0.00      0.00         0
        2880       0.00      0.00      0.00         1
        3240       0.00      0.00      0.00         1
        3360       0.00      0.00      0.00         1
        4159       0.00      0.00      0.00         1

    accuracy                           0.17         6
   macro avg       0.04      0.12      0.06         6
weighted avg       0.06      0.17      0.08         6



C:\Users\Guga\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guga\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guga\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Guga\miniconda3\lib\site-packages\sklearn\metrics\_cla

In [234]:
#Predição para a safra 2006
classifier.predict(concat_data.iloc[6:7,:])

array(['2280'], dtype=object)